In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import glob
import os

plt.style.use('ggplot') # Make the graphs a bit prettier

In [2]:
data = {}

for path in glob.glob(os.path.expanduser("~/dev/ma/data/results/pre_dlm1/*.json")):
  for typename, items in json.load(open(path)).items():
    if typename in data:
      data[typename].update(items)
    else:
      data[typename] = items

frames = { typename: pd.DataFrame.from_dict(items, orient='index') for typename, items in data.items() }

graph_names = {
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_100000-sorted-preprocessed-*.bin': 'LFR 100K', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_1000000-preprocessed-*.bin': 'LFR 1M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_10000000-preprocessed-*.bin': 'LFR 10M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_100000000-preprocessed-*.bin': 'LFR 100M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_200000000-sorted-preprocessed-*.bin': 'LFR 200M', 
    '/home/kit/iti/kp0036/graphs/hypercubegraph23-preprocessed-*.bin': 'hypercube', 
    '/home/kit/iti/kp0036/graphs/uk-2002.metis-preprocessed-*.bin': 'uk-2002', 
    '/home/kit/iti/kp0036/graphs/uk-2007-05.metis-preprocessed-*.bin': 'uk-2007-05', 
    '/home/kit/iti/kp0036/graphs/in-2004.metis-preprocessed-*.bin': 'in-2004', 
    '/home/kit/iti/kp0036/graphs/com-friendster-preprocessed-*.bin': 'com-friendster', 
    '/home/kit/iti/kp0036/graphs/com-lj.ungraph-preprocessed-*.bin': 'com-lj', 
    '/home/kit/iti/kp0036/graphs/com-orkut.ungraph-preprocessed-*.bin': 'com-orkut', 
    '/home/kit/iti/kp0036/graphs/com-youtube.ungraph-preprocessed-*.bin': 'com-youtube', 
    '/home/kit/iti/kp0036/graphs/com-amazon.ungraph-preprocessed-*.bin': 'com-amazon'
}


frames['program_run'].replace({ 'graph': graph_names }, inplace=True)

In [3]:
frames['clustering'] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'], left_on='program_run_id', right_index=True) \
    .groupby(['graph', 'local_moving_stopping_criterion', 'local_moving_node_ratio'])['modularity', 'runtime'] \
    .mean()

modularity  \
graph      local_moving_stopping_criterion local_moving_node_ratio               
LFR 10M    cluster_count                   1                          0.587054   
                                           2                          0.595544   
                                           4                          0.587416   
                                           8                          0.597741   
                                           dynamic                    0.578653   
           moved_count                     1                          0.591836   
                                           2                          0.596406   
                                           4                          0.597009   
                                           8                          0.589233   
                                           dynamic                    0.591084   
friendster cluster_count                   1                          0.503160   
                                           2                          0.562513   
                                           4                          0.577368   
                                           8                          0.592235   
                                           dynamic                    0.606859   
           moved_count                     2                          0.591695   
                                           4                          0.609500   
                                           8                          0.613189   
                                           dynamic                    0.625279   
uk-2002    cluster_count                   1                          0.351908   
                                           2                          0.988926   
                                           4                          0.989308   
                                           8                          0.989551   
                                           dynamic                    0.988914   
           moved_count                     2                          0.989319   
                                           4                          0.989561   
                                           8                          0.989670   
                                           dynamic                    0.989531   

                                                                        runtime  
graph      local_moving_stopping_criterion local_moving_node_ratio               
LFR 10M    cluster_count                   1                         124.740916  
                                           2                         186.339220  
                                           4                         289.715450  
                                           8                         914.074681  
                                           dynamic                   199.217159  
           moved_count                     1                         240.974209  
                                           2                         167.030770  
                                           4                         173.034491  
                                           8                         209.436321  
                                           dynamic                   206.705705  
friendster cluster_count                   1                         762.548029  
                                           2                         999.474926  
                                           4                        1315.349956  
                                           8                        2911.561037  
                                           dynamic                  1461.522005  
           moved_count                     2                        1866.585136  
                                           4                        2556.930383  
                                           8     

In [16]:
df = frames['clustering'] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'], left_on='program_run_id', right_index=True) \
    .groupby(['graph', 'local_moving_stopping_criterion', 'local_moving_node_ratio'])['modularity', 'runtime'] \
    .mean() \
    .stack() \
    .unstack(['local_moving_stopping_criterion', 'local_moving_node_ratio'])
  
df.columns = df.columns.rename(['stopping criterion', 'ratio'])

df

stopping criterion    cluster_count                                        \
ratio                             1           2            4            8   
graph                                                                       
LFR 10M    modularity      0.587054    0.595544     0.587416     0.597741   
           runtime       124.740916  186.339220   289.715450   914.074681   
friendster modularity      0.503160    0.562513     0.577368     0.592235   
           runtime       762.548029  999.474926  1315.349956  2911.561037   
uk-2002    modularity      0.351908    0.988926     0.989308     0.989551   
           runtime        85.967360  133.125686   154.490371   462.429317   

stopping criterion                 moved_count                            \
ratio                      dynamic           1            2            4   
graph                                                                      
LFR 10M    modularity     0.578653    0.591836     0.596406     0.597009   
           runtime      199.217159  240.974209   167.030770   173.034491   
friendster modularity     0.606859         NaN     0.591695     0.609500   
           runtime     1461.522005         NaN  1866.585136  2556.930383   
uk-2002    modularity     0.988914         NaN     0.989319     0.989561   
           runtime       66.893048         NaN   128.487342   116.285455   

stopping criterion                               
ratio                            8      dynamic  
graph                                            
LFR 10M    modularity     0.589233     0.591084  
           runtime      209.436321   206.705705  
friendster modularity     0.613189     0.625279  
           runtime     2290.740927  2731.077903  
uk-2002    modularity     0.989670     0.989531  
           runtime      113.849404   121.070345

In [17]:
deltas = df.copy()
for c in df.columns:
    deltas[c] = sum([(df[c] - df[d].fillna(df[c])) for d in df.columns])
deltas

stopping criterion    cluster_count                                         \
ratio                             1            2            4            8   
graph                                                                        
LFR 10M    modularity     -0.041436     0.043464    -0.037816     0.065434   
           runtime     -1463.859762  -847.876722   185.885578  6429.477888   
friendster modularity     -0.753358    -0.219181    -0.085486     0.048317   
           runtime    -10032.858041 -7900.515968 -5057.640698  9308.259031   
uk-2002    modularity     -5.099516     0.633646     0.637084     0.639271   
           runtime      -608.892088  -184.467154     7.815011  2779.265525   

stopping criterion                 moved_count                            \
ratio                      dynamic           1            2            4   
graph                                                                      
LFR 10M    modularity    -0.125446    0.006384     0.052084     0.058114   
           runtime     -719.097332 -301.526832 -1040.961222  -980.924012   
friendster modularity     0.179933         NaN     0.043457     0.203702   
           runtime    -3742.092257         NaN   -96.524078  6116.583145   
uk-2002    modularity     0.633538         NaN     0.637183     0.639361   
           runtime     -780.560896         NaN  -226.212250  -336.029233   

stopping criterion                               
ratio                            8      dynamic  
graph                                            
LFR 10M    modularity    -0.019646    -0.001136  
           runtime     -616.905712  -644.211872  
friendster modularity     0.236903     0.345713  
           runtime     3720.878041  7683.910825  
uk-2002    modularity     0.640342     0.639091  
           runtime     -357.953692  -292.965223